# Proyecto final de Aprendizaje Automático
## Autor:
Daniel Useche.
ignaciousecheparedes@gmail.com

Raul Saavedra
maul.saavedrarunet.edu.veolea@gmail.com

---
## Descripción
Aplicación de multiclasificadores tipo Bagging y Boosting a la data de pacientes diabéticos extraida de 130 hospitales de los Estados Unidos en el periodo comprendido entre 1999-2008.

El data set original puede encontrarse en el siguiente [enlace](https://archive.ics.uci.edu/ml/datasets/Diabetes+130-US+hospitals+for+years+1999-2008#) o en la dirección: https://archive.ics.uci.edu/ml/datasets/Diabetes+130-US+hospitals+for+years+1999-2008#

Para una explicación detallada de los atributos presentes en el dataset dirigirse a https://www.hindawi.com/journals/bmri/2014/781670/

---

## Cargando el DataSet 

In [ ]:
import numpy as np
import pandas as pd 
diabeticDataSet = pd.read_csv("diabetic_data.csv", header = 0)
diabeticDataSet.tail()

---
## Preprocesado:

### Eliminación de caracteristicas ruidosas, o con valores faltantes:
Como todo conjunto de datos, el data set utilizado tiene atributos que poseen elementos faltantes en su mayoria, redundantes o ruidosos, como se muestra en la tabla 1 (pág 3) del [documento](https://www.hindawi.com/journals/bmri/2014/781670/) que describe los atributos, los siguientes atributos deben ser depurados del dataset por su cantidad de valores faltantes:
- weight: indica el peso del paciente, pero como su medida no es obligatoria, posee un aproximado de 97% de valores faltantes.
- payer_code: número que indica el tipo de pago realizado por el paciente, 40% de datos faltantes e irrelevante para el análisis.
- medical_specialty: indica la especialidad del médico que brindó la atención, posee 49% de datos faltantes.

In [ ]:
diabeticDataSet = diabeticDataSet.drop(columns = ['weight', 'payer_code', 'medical_specialty'])

### Estableciendo un registro único por paciente:
Tambien hay que tomar en cuenta que puede existir mas de un registro para el mismo paciente, por lo que se tomará el atributo **patient_nbr** para filtrar los encuentros y así no existan pacientes duplicados:

In [ ]:
diabeticDataSet = diabeticDataSet.drop_duplicates(subset='patient_nbr')

### Vista preliminar del dataset:

In [ ]:
diabeticDataSet.T

### Transformación de valores faltantes con el formato '?' a un valor compatible Nan
La Mayoría de los algoritmos de aprendizaje requieren que sus entradas no posean valores faltantes, y no funcionarán si este requerimiento es violado [ref](https://scikit-learn.org/stable/modules/impute.html#impute).

In [ ]:
diabeticDataSet = diabeticDataSet.replace('?', np.nan) 

In [ ]:
diabeticDataSet.gender.value_counts()
